In [2]:
import xarray as xr
import os
import pathlib 
#from boia_domain_ocpt_karoon_blocos import CreateMapBoia
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from shapely.geometry import Point
from shapely.prepared import prep
from shapely.ops import unary_union
import matplotlib 
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.patheffects as pe
import matplotlib.ticker as mticker
import matplotlib.patches as patches

import windrose 
from matplotlib.gridspec import GridSpec
import matplotlib.ticker as ticker
import pandas as pd
import cmocean

from scipy.interpolate import griddata

import geopandas as gpd

In [3]:
def cmap_wave():
     # Lista de cores em hexadecimal
    linear_gradient = [
        (144, 240, 242), # azul clarin
        (71, 221, 230), # azul claro 
        (49, 145, 176), # azul  
        (53, 102, 175), # azul escurao
        (173, 71, 173), # lilas
        (126, 49, 59), # vinho 
        (192, 123, 111),
        (192, 162, 157),
    ]

    colors_rgb_normalized = [(r/255, g/255, b/255) for r, g, b in linear_gradient]


    cmapwave = LinearSegmentedColormap.from_list("my_colormap", colors_rgb_normalized)

    return cmapwave 

def create_output_directory(base_dir, test_name):
    PDIR = f'{base_dir}/{test_name}/'
    pasta = pathlib.Path(PDIR)
    ind = 1
    while pasta.exists():
        PDIR = f'{base_dir}/{test_name}_{ind}/'
        pasta = pathlib.Path(PDIR)
        ind += 1
    pasta.mkdir(parents=True, exist_ok=False)
    return PDIR

In [5]:
waves = xr.open_dataset('C:\\Users\\matheus\\Desktop\\ZEEC\\data\\waverys_zeec_2022.nc')
marem_path = "data/MAREM/"
clname = 'BR_MAREM_OP_pol_v2'
bs_gdf = gpd.read_file(marem_path + f'{clname}.shp')

c:\Users\matheus\miniconda3\envs\clim_meq\Lib\site-packages\pyogrio\core.py:35: RuntimeWarning: Could not detect GDAL data files.  Set GDAL_DATA environment variable to the correct path.
  _init_gdal_data()
c:\Users\matheus\miniconda3\envs\clim_meq\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: data/MAREM/BR_MAREM_OP_pol_v2.shp contains polygon(s) with rings with invalid winding order. Autocorrecting them, but that shapefile should be corrected using ogr2ogr for example.
  return ogr_read(


In [6]:
#hs = waves['VHM0'] 

#hs = hs.sel(time = slice('2022-06-01', '2022-08-31'))

waves_JJA = waves.groupby('time.season').mean('time').sel(season = 'JJA')

waves_JJA

<xarray.Dataset> Size: 7kB
Dimensions:    (latitude: 18, longitude: 30)
Coordinates:
  * latitude   (latitude) float32 72B -21.4 -21.2 -21.0 ... -18.4 -18.2 -18.0
  * longitude  (longitude) float32 120B -41.8 -41.6 -41.4 ... -36.4 -36.2 -36.0
    season     <U3 12B 'JJA'
Data variables:
    VMDR       (latitude, longitude) float32 2kB nan nan nan ... 136.3 135.1
    VTPK       (latitude, longitude) float32 2kB nan nan nan ... 11.22 11.11
    VHM0       (latitude, longitude) float32 2kB nan nan nan ... 2.05 2.05 2.077
Attributes:
    Conventions:       CF-1.11
    title:             Mean fields from global wave model MFWAM of Meteo-Fran...
    institution:       METEO-FRANCE
    producer:          CMEMS - Global Monitoring and Forecasting Centre
    credit:            E.U. Copernicus Marine Service Information (CMEMS)
    contact:           servicedesk.cmems@mercator-ocean.eu
    references:        http://marine.copernicus.eu
    subset:source:     ARCO data downloaded from the Marine Data Store using ...
    subset:productId:  GLOBAL_MULTIYEAR_WAV_001_032
    subset:datasetId:  cmems_mod_glo_wav_my_0.2deg_PT3H-i_202411
    subset:date:       2025-01-31T17:44:33.190Z

In [12]:


cmap1 = cmap_wave()
# create a plot 2x6 to show all months 
sp_quiver_onda = 2
levels_onda = [0.2, 0.4, 0.6, 0.8, 1, 1.25, 1.5, 1.75, 2, 2.25, 2.5, 2.75, 3]   # Padronizar níveis de altura significativa
tags = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l']
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(17, 8), subplot_kw={'projection': ccrs.PlateCarree()})

    
direcao = waves_JJA['VMDR']
hs = waves_JJA['VHM0']
period = waves_JJA['VTPK']

hs_interpolado = hs.interpolate_na(dim='longitude', method='linear', fill_value="extrapolate").interpolate_na(dim='latitude', method='linear', fill_value="extrapolate")

u_onda = -np.sin(np.deg2rad(direcao))
v_onda = -np.cos(np.deg2rad(direcao))

# Altura significativa e direção média de onda
hsplot = ax.contourf(waves_JJA.longitude, waves_JJA.latitude, hs_interpolado, levels=levels_onda, extend='both', cmap=cmap1)
levels_period = [2, 3, 4, 5, 6, 7, 8, 9]
period_contour = ax.contour(waves_JJA.longitude, waves_JJA.latitude, period, levels=levels_period, colors='white', linewidths=1)
#ax.clabel(period_contour, fontsize=9, inline=True, fmt='%1i s')
ax.quiver(u_onda.longitude[::sp_quiver_onda], u_onda.latitude[::sp_quiver_onda], u_onda[::sp_quiver_onda, ::sp_quiver_onda], v_onda[::sp_quiver_onda, ::sp_quiver_onda], scale=18,     width=0.003,       # Largura das setas (mais fino)
    headwidth=3,       # Largura da cabeça da seta
    headlength=3.5,      # Comprimento da cabeça da seta
    headaxislength=2.5)
    #ds_prec_monthly.isel(time=i).precipitation.plot.contourf(ax=ax, levels=clevs, cmap=cmap, norm=norm, extend='both')
ax.set_title('Média JJA - 2022', fontsize=15, fontweight='bold', color='#3A3A3A')
#ax.coastlines()
gl = ax.gridlines(draw_labels=True, alpha=0.75, linestyle='--')
gl.top_labels = False
gl.right_labels = False
gl.xlabel_style = {'fontsize': 12, 'fontweight': 'bold'}
gl.ylabel_style = {'fontsize': 12, 'fontweight': 'bold'}

gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.xlocator = mticker.FixedLocator(np.arange(int(min(waves_JJA.longitude)), int(max(waves_JJA.longitude)) + 1, 2.5))


ax.add_feature(cfeature.LAND.with_scale('10m'), facecolor='#F5DEB3', alpha=1, zorder=13)

# Adicione o shapefile da linha de costa:
ax.add_geometries(
    bs_gdf.geometry,
    crs=ccrs.PlateCarree(),
    facecolor='none',
    edgecolor='black',
    linewidth=0.8,
    zorder=25
)


plt.subplots_adjust(top=0.97,bottom=0.01, left=0.1, right=0.90, hspace=0.115, wspace=0.025)
cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])
cbar = fig.colorbar(ax.collections[0], cax=cbar_ax, extend='both', orientation='vertical')
cbar.set_label('Altura Significativa de Onda [m]', fontsize=14, fontweight='bold')

#setting values of labels in colorbar as bold 
for label in cbar.ax.get_yticklabels():
    label.set_fontsize(12)
    label.set_fontweight('bold')



plt.savefig('JJA_waverys_2022_ES.png', dpi=450, bbox_inches='tight')
plt.close()